In [1]:
import os
import numpy as np
import librosa

In [2]:
from math import floor
from lib_augment import load_and_normalize
from lib_feature import feature_whistleness

%matplotlib inline
import matplotlib.pyplot as plt


In [3]:
# deploy_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2000_48kHz'
species_dict = {'BD': 0, 'CD': 1, 'STR': 2, 'SPT': 3, 'SPIN': 4, 'PLT': 5, 'RT': 6,  'FKW': 7}
fs = 48000
time_reso = 0.02
hop_length = int(time_reso*fs)  #
species_list = list(species_dict.keys())
species_id = list(species_dict.values())

In [4]:
species_to_code = {'bottlenose': 'BD', 'longbeaked_common': 'CD', 'shortbeaked_common': 'CD', 'common': 'CD',
                   'striped': 'STR', 'spotted': 'SPT', 'spinner': 'SPIN', 'pilot': 'PLT', 'roughtoothed': 'RT',
                   'false_killer': 'FKW'}

In [5]:
# species_dict = {'BD': 0, 'CD': 1, 'STR': 2, 'SPT': 3, 'SPIN': 4, 'PLT': 5, 'RT': 6,  'FKW': 7}

step_sec = 1.0  # sec
whistle_ratio = 0.2  # percentage of whislteness frames in a long window
# step_size = int(1.0*fs)
win_long_sec = 10   # 10 sec
step_ind = int(win_long_sec/step_sec)
step_size = int(step_sec*fs)
win_size = int(2.0*fs)

In [6]:
# loading the trained model
from tensorflow.keras.models import load_model
import tensorflow_addons.layers.spatial_pyramid_pooling as spp


model_whistleness_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__fit_result_whistleness/2021-02-02_200514_resnet18_expt_alldata_run0_f1_lr_0.001/epoch_163_valloss_0.2434_valacc_0.9472.hdf5'
model_whistleness = load_model(model_whistleness_path)
model_whistleness.summary()

# model_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__fit_result_species/20210303_233728/STAR2000/epoch_53_valloss_0.9505_valacc_0.7552.hdf5'
# model_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__fit_result_species/20210305_105231/STAR2000/epoch_56_valloss_0.1945_valacc_0.7305.hdf5'
model_species_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__fit_result_species/20210310_095616/STAR2000/epoch_48_valloss_0.1938_valacc_0.7403.hdf5'

model_species = load_model(model_species_path)
model_species.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 128, 1) 0                                            
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 100, 128, 16) 144         input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 100, 128, 16) 64          conv0[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, 100, 128, 16) 0           bn_conv1[0][0]                   
______________________________________________________________________________________________

In [7]:
def enounter_features(encounter_folder, win_size, stpe_size):
    sound_files = os.listdir(encounter_folder)
    sound_files.sort()
    
    spec_feas_curr = []
    samples_tot = 0
    for ff in sound_files:
    # for ff in sound_files[:2]:
    # for ff in [sound_files[1]]:
        print('=='+ff)
        samples, _ = librosa.load(os.path.join(encounter_folder, ff), sr=fs)
        # print('Sound length: '+str(samples.shape[0]/fs))
        samples_tot += samples.shape[0]
        if samples.ndim >= 2:
            samples = samples[:, 0]

        for ss in range(floor((samples.shape[0]-win_size)/step_size)+1):
            samples_curr = samples[ss*step_size:ss*step_size+win_size]
            samples_curr = samples_curr - samples_curr.mean()

            spectro = librosa.feature.melspectrogram(samples_curr,sr=fs, hop_length=960, power=1.0)
            # spectro = librosa.feature.melspectrogram(samples_curr,sr=fs)
            spec_feas_curr.append(feature_whistleness(spectro))
    sec_tot  = samples_tot/fs
    print('Total sound length: '+str(sec_tot))
    spec_feas = np.stack(spec_feas_curr)
    spec_feas = np.expand_dims(spec_feas[:, :100, :], axis=3)
    
    # score = model_species.predict(spec_feas)
    
    return spec_feas, sec_tot

In [8]:
np.set_printoptions(linewidth=200, precision=2, suppress=True)

## Test on STAR2000 & train on the others

In [9]:
sound_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald'
sound_deploy = ['STAR2000_48kHz', 'STAR2003_48kHz', 'STAR2006_48kHz', 'HICEAS2002_48kHz', 'PICEAS2005_48kHz']

In [10]:
dd = 0
deploy_path = os.path.join(sound_path, sound_deploy[dd])
encounter_test0 =  os.listdir(deploy_path)
encounter_test0.sort()
encounter_test = [os.path.join(deploy_path, ee) for ee in encounter_test0]

In [11]:
encounter_test

['/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2000_48kHz/bottlenose s352',
 '/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2000_48kHz/bottlenose s46',
 '/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2000_48kHz/longbeaked_common s329',
 '/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2000_48kHz/longbeaked_common s338',
 '/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2000_48kHz/pilot s288',
 '/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2000_48kHz/roughtoothed s417',
 '/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2000_48kHz/shortbeaked_common s282',
 '/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2000_48kHz/shortbeaked_common s302',
 '/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48

In [12]:
encounter_test[14]

'/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2000_48kHz/shortbeaked_common s376'

In [13]:
### run on a encounter encounter[0]

spec_feas_train = []
spec_feas_test = []
label_train = []
label_test = []
for ee in encounter_test:
# for ee in [encounter_test[14]]:
# for ee in encounter_test[:2]:
# for ee in ['shortbeaked_common s628']:
    print(ee)

    # encounter_folder = os.path.join(deploy_path_test, ee)
    spec_feas, sec_tot = enounter_features(ee, win_size, step_size)
    
    whistleness = model_whistleness.predict(spec_feas)
    whistleness_bin = whistleness[:, 1]>0.9
    num_whistle = np.where(whistleness_bin)[0].shape[0]
    num_frames = whistleness.shape[0]
    print('Number of whistleness frames: '+str(num_whistle)+'/'+str(num_frames)+'('+'str(num_whistle/num_frames)'+')')    
    scores = model_species.predict(spec_feas)
    
    
    # For each 10 sec / 100 frames
    # for ii in range(int(floor(num_frames*1.0/100))):
    for ii in range(num_frames-step_ind):
        score_curr = scores[ii:ii+step_ind]
        whistle_bin_curr = whistleness[ii:ii+step_ind, 1]>0.9
        num_whistleness_curr = np.where(whistle_bin_curr)[0].shape[0]
        # print('Whistleness:')
        # print(num_whistleness_curr)
        
        if num_whistleness_curr / step_ind>= whistle_ratio:  # whistleness over 2 sec
            spec_feas_long = np.stack(spec_feas[ii:ii+step_ind, :, :, :])
            label_long = species_to_code[os.path.basename(ee).split(' ')[0]]
            
            spec_feas_train.append(spec_feas_long)
            label_train.append(label_long)
            if ii % step_ind ==  0:
                spec_feas_test.append(spec_feas_long)
                label_test.append(label_long)
                
    print('Number of training data samples: '+str(len(spec_feas_train))+'\n')

/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2000_48kHz/bottlenose s352
==Acst0953.b02.wav
==Acst0954.b02.wav
==Acst0955.b02.wav
==Acst0956.b02.wav
==Acst0957.b02.wav
Total sound length: 282.33804166666664
Number of whistleness frames: 152/275(str(num_whistle/num_frames))
Number of training data samples: 247

/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2000_48kHz/bottlenose s46
==ACST1811.810.wav
==ACST1829.810.wav
==ACST1830.810.wav
==ACST1832.810.wav
==ACST1833.810.wav
==ACST1835.810.wav
==ACST1836.810.wav
Total sound length: 351.929375
Number of whistleness frames: 83/343(str(num_whistle/num_frames))
Number of training data samples: 408

/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2000_48kHz/longbeaked_common s329
==Acst1200.a30.wav
==Acst1202.a30.wav
==Acst1203.a30.wav
==Acst1204.a30.wav
==Acst1206.a30.wav
==Acst1208.a30.wav
==Acst1211.a30.wav
==Acst1214.a30.wav
==Acs

In [14]:
feature_train = np.stack(spec_feas_train)
del spec_feas_train
label_train = np.array(label_train)
feature_test = np.stack(spec_feas_test)
del spec_feas_test
label_test = np.array(label_test)

In [15]:
ten_sec_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__dataset/ten_seconds_data'
np.savez(os.path.join(ten_sec_path, sound_deploy[dd]), feature_train=feature_train, label_train=label_train, feature_test=feature_test, label_test=label_test)

## STAR2003

In [16]:
dd = 1
deploy_path = os.path.join(sound_path, sound_deploy[dd])
encounter_test0 =  os.listdir(deploy_path)
encounter_test0.sort()
encounter_test = [os.path.join(deploy_path, ee) for ee in encounter_test0]

In [17]:
### run on a encounter encounter[0]

spec_feas_train = []
spec_feas_test = []
label_train = []
label_test = []
for ee in encounter_test:
# for ee in [encounter_test[14]]:
# for ee in encounter_test[:2]:
# for ee in ['shortbeaked_common s628']:
    print(ee)

    # encounter_folder = os.path.join(deploy_path_test, ee)
    spec_feas, sec_tot = enounter_features(ee, win_size, step_size)
    
    whistleness = model_whistleness.predict(spec_feas)
    whistleness_bin = whistleness[:, 1]>0.9
    num_whistle = np.where(whistleness_bin)[0].shape[0]
    num_frames = whistleness.shape[0]
    print('Number of whistleness frames: '+str(num_whistle)+'/'+str(num_frames)+'('+'str(num_whistle/num_frames)'+')')    
    scores = model_species.predict(spec_feas)
    
    
    # For each 10 sec / 100 frames
    # for ii in range(int(floor(num_frames*1.0/100))):
    for ii in range(num_frames-step_ind):
        score_curr = scores[ii:ii+step_ind]
        whistle_bin_curr = whistleness[ii:ii+step_ind, 1]>0.9
        num_whistleness_curr = np.where(whistle_bin_curr)[0].shape[0]
        # print('Whistleness:')
        # print(num_whistleness_curr)
        
        if num_whistleness_curr / step_ind>= whistle_ratio:  # whistleness over 2 sec
            spec_feas_long = np.stack(spec_feas[ii:ii+step_ind, :, :, :])
            label_long = species_to_code[os.path.basename(ee).split(' ')[0]]
            
            spec_feas_train.append(spec_feas_long)
            label_train.append(label_long)
            if ii % step_ind ==  0:
                spec_feas_test.append(spec_feas_long)
                label_test.append(label_long)
                
    print('Number of training data samples: '+str(len(spec_feas_train))+'\n')

/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2003_48kHz/bottlenose s494
==STAR031106-113842.wav
Total sound length: 36.05
Number of whistleness frames: 8/35(str(num_whistle/num_frames))
Number of training data samples: 15

/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2003_48kHz/bottlenose s516
==STAR031109-114625.wav
==STAR031109-114829.wav
==STAR031109-114858.wav
==STAR031109-114921.wav
==STAR031109-115000.wav
==STAR031109-115149.wav
==STAR031109-115201.wav
==STAR031109-115500.wav
Total sound length: 433.8
Number of whistleness frames: 157/422(str(num_whistle/num_frames))
Number of training data samples: 338

/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2003_48kHz/bottlenose s586
==STAR031120-110007.wav
==STAR031120-110203.wav
==STAR031120-110500.wav
==STAR031120-110639.wav
==STAR031120-110812.wav
==STAR031120-111000.wav
==STAR031120-111050.wav
==STAR031120-111231.wav
==ST

In [18]:
feature_train = np.stack(spec_feas_train)
del spec_feas_train
label_train = np.array(label_train)
feature_test = np.stack(spec_feas_test)
del spec_feas_test
label_test = np.array(label_test)

In [19]:
ten_sec_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__dataset/ten_seconds_data'
np.savez(os.path.join(ten_sec_path, sound_deploy[dd]), feature_train=feature_train, label_train=label_train, feature_test=feature_test, label_test=label_test)

## STAR2006

In [20]:
dd = 2
deploy_path = os.path.join(sound_path, sound_deploy[dd])
encounter_test0 =  os.listdir(deploy_path)
encounter_test0.sort()
encounter_test = [os.path.join(deploy_path, ee) for ee in encounter_test0]

In [21]:
### run on a encounter encounter[0]

spec_feas_train = []
spec_feas_test = []
label_train = []
label_test = []
for ee in encounter_test:
# for ee in [encounter_test[14]]:
# for ee in encounter_test[:2]:
# for ee in ['shortbeaked_common s628']:
    print(ee)

    # encounter_folder = os.path.join(deploy_path_test, ee)
    spec_feas, sec_tot = enounter_features(ee, win_size, step_size)
    
    whistleness = model_whistleness.predict(spec_feas)
    whistleness_bin = whistleness[:, 1]>0.9
    num_whistle = np.where(whistleness_bin)[0].shape[0]
    num_frames = whistleness.shape[0]
    print('Number of whistleness frames: '+str(num_whistle)+'/'+str(num_frames)+'('+'str(num_whistle/num_frames)'+')')    
    scores = model_species.predict(spec_feas)
    
    
    # For each 10 sec / 100 frames
    # for ii in range(int(floor(num_frames*1.0/100))):
    for ii in range(num_frames-step_ind):
        score_curr = scores[ii:ii+step_ind]
        whistle_bin_curr = whistleness[ii:ii+step_ind, 1]>0.9
        num_whistleness_curr = np.where(whistle_bin_curr)[0].shape[0]
        # print('Whistleness:')
        # print(num_whistleness_curr)
        
        if num_whistleness_curr / step_ind>= whistle_ratio:  # whistleness over 2 sec
            spec_feas_long = np.stack(spec_feas[ii:ii+step_ind, :, :, :])
            label_long = species_to_code[os.path.basename(ee).split(' ')[0]]
            
            spec_feas_train.append(spec_feas_long)
            label_train.append(label_long)
            if ii % step_ind ==  0:
                spec_feas_test.append(spec_feas_long)
                label_test.append(label_long)
                
    print('Number of training data samples: '+str(len(spec_feas_train))+'\n')

/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2006_48kHz/false_killer s128
==STAR06-091006-1125.wav
==STAR06-091006-1130.wav
==STAR06-091006-1135.wav
==STAR06-091006-1140.wav
==STAR06-091006-1145.wav
==STAR06-091006-1150.wav
==STAR06-091006-1155.wav
==STAR06-091006-1156.wav
==STAR06-091006-1200.wav
==STAR06-091006-1205.wav
==STAR06-091006-1210.wav
==STAR06-091006-1215.wav
==STAR06-091006-1220.wav
Total sound length: 3600.037
Number of whistleness frames: 1741/3586(str(num_whistle/num_frames))
Number of training data samples: 2729

/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2006_48kHz/pilot s142
==STAR06-091606-0820.wav
==STAR06-091606-0825.wav
==STAR06-091606-0830.wav
==STAR06-091606-0835.wav
==STAR06-091606-0840.wav
Total sound length: 1500.0
Number of whistleness frames: 25/1495(str(num_whistle/num_frames))
Number of training data samples: 2748

/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/_

In [22]:
feature_train = np.stack(spec_feas_train)
del spec_feas_train
label_train = np.array(label_train)
feature_test = np.stack(spec_feas_test)
del spec_feas_test
label_test = np.array(label_test)

In [23]:
ten_sec_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__dataset/ten_seconds_data'
np.savez(os.path.join(ten_sec_path, sound_deploy[dd]), feature_train=feature_train, label_train=label_train, feature_test=feature_test, label_test=label_test)

## HIPEAS

In [24]:
dd = 3
deploy_path = os.path.join(sound_path, sound_deploy[dd])
encounter_test0 =  os.listdir(deploy_path)
encounter_test0.sort()
encounter_test = [os.path.join(deploy_path, ee) for ee in encounter_test0]

In [25]:
### run on a encounter encounter[0]

spec_feas_train = []
spec_feas_test = []
label_train = []
label_test = []
for ee in encounter_test:
# for ee in [encounter_test[14]]:
# for ee in encounter_test[:2]:
# for ee in ['shortbeaked_common s628']:
    print(ee)

    # encounter_folder = os.path.join(deploy_path_test, ee)
    spec_feas, sec_tot = enounter_features(ee, win_size, step_size)
    
    whistleness = model_whistleness.predict(spec_feas)
    whistleness_bin = whistleness[:, 1]>0.9
    num_whistle = np.where(whistleness_bin)[0].shape[0]
    num_frames = whistleness.shape[0]
    print('Number of whistleness frames: '+str(num_whistle)+'/'+str(num_frames)+'('+'str(num_whistle/num_frames)'+')')    
    scores = model_species.predict(spec_feas)
    
    
    # For each 10 sec / 100 frames
    # for ii in range(int(floor(num_frames*1.0/100))):
    for ii in range(num_frames-step_ind):
        score_curr = scores[ii:ii+step_ind]
        whistle_bin_curr = whistleness[ii:ii+step_ind, 1]>0.9
        num_whistleness_curr = np.where(whistle_bin_curr)[0].shape[0]
        # print('Whistleness:')
        # print(num_whistleness_curr)
        
        if num_whistleness_curr / step_ind>= whistle_ratio:  # whistleness over 2 sec
            spec_feas_long = np.stack(spec_feas[ii:ii+step_ind, :, :, :])
            label_long = species_to_code[os.path.basename(ee).split(' ')[0]]
            
            spec_feas_train.append(spec_feas_long)
            label_train.append(label_long)
            if ii % step_ind ==  0:
                spec_feas_test.append(spec_feas_long)
                label_test.append(label_long)
                
    print('Number of training data samples: '+str(len(spec_feas_train))+'\n')

/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/HICEAS2002_48kHz/bottlenose s165
==HICEAS020908-084843.wav
==HICEAS020908-084943.wav
==HICEAS020908-085043.wav
==HICEAS020908-085143.wav
==HICEAS020908-090007.wav
==HICEAS020908-090107.wav
==HICEAS020908-090207.wav
==HICEAS020908-090307.wav
Total sound length: 403.14
Number of whistleness frames: 73/394(str(num_whistle/num_frames))
Number of training data samples: 177

/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/HICEAS2002_48kHz/pilot s219
==HICEAS021017-112120.wav
==HICEAS021017-112220.wav
==HICEAS021017-112320.wav
==HICEAS021017-112420.wav
Total sound length: 183.28
Number of whistleness frames: 11/179(str(num_whistle/num_frames))
Number of training data samples: 208

/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/HICEAS2002_48kHz/pilot s228
==HICEAS021018-104519.wav
==HICEAS021018-104619.wav
==HICEAS021018-104719.wav
==HICEAS021018-104819.

In [26]:
feature_train = np.stack(spec_feas_train)
del spec_feas_train
label_train = np.array(label_train)
feature_test = np.stack(spec_feas_test)
del spec_feas_test
label_test = np.array(label_test)

In [27]:
ten_sec_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__dataset/ten_seconds_data'
np.savez(os.path.join(ten_sec_path, sound_deploy[dd]), feature_train=feature_train, label_train=label_train, feature_test=feature_test, label_test=label_test)

## PICEAS

In [28]:
dd = 4
deploy_path = os.path.join(sound_path, sound_deploy[dd])
encounter_test0 =  os.listdir(deploy_path)
encounter_test0.sort()
encounter_test = [os.path.join(deploy_path, ee) for ee in encounter_test0]

In [29]:
### run on a encounter encounter[0]

spec_feas_train = []
spec_feas_test = []
label_train = []
label_test = []
for ee in encounter_test:
# for ee in [encounter_test[14]]:
# for ee in encounter_test[:2]:
# for ee in ['shortbeaked_common s628']:
    print(ee)

    # encounter_folder = os.path.join(deploy_path_test, ee)
    spec_feas, sec_tot = enounter_features(ee, win_size, step_size)
    
    whistleness = model_whistleness.predict(spec_feas)
    whistleness_bin = whistleness[:, 1]>0.9
    num_whistle = np.where(whistleness_bin)[0].shape[0]
    num_frames = whistleness.shape[0]
    print('Number of whistleness frames: '+str(num_whistle)+'/'+str(num_frames)+'('+'str(num_whistle/num_frames)'+')')    
    scores = model_species.predict(spec_feas)
    
    
    # For each 10 sec / 100 frames
    # for ii in range(int(floor(num_frames*1.0/100))):
    for ii in range(num_frames-step_ind):
        score_curr = scores[ii:ii+step_ind]
        whistle_bin_curr = whistleness[ii:ii+step_ind, 1]>0.9
        num_whistleness_curr = np.where(whistle_bin_curr)[0].shape[0]
        # print('Whistleness:')
        # print(num_whistleness_curr)
        
        if num_whistleness_curr / step_ind>= whistle_ratio:  # whistleness over 2 sec
            spec_feas_long = np.stack(spec_feas[ii:ii+step_ind, :, :, :])
            label_long = species_to_code[os.path.basename(ee).split(' ')[0]]
            
            spec_feas_train.append(spec_feas_long)
            label_train.append(label_long)
            if ii % step_ind ==  0:
                spec_feas_test.append(spec_feas_long)
                label_test.append(label_long)
                
    print('Number of training data samples: '+str(len(spec_feas_train))+'\n')

/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/PICEAS2005_48kHz/bottlenose a101
==piceas-050824-080400.wav
==piceas-050824-080600.wav
==piceas-050824-080800.wav
==piceas-050824-081000.wav
==piceas-050824-081200.wav
==piceas-050824-081400.wav
==piceas-050824-081600.wav
==piceas-050824-081800.wav
==piceas-050824-082000.wav
==piceas-050824-082200.wav
==piceas-050824-082400.wav
==piceas-050824-082600.wav
==piceas-050824-082648.wav
==piceas-050824-082800.wav
==piceas-050824-083000.wav
==piceas-050824-083200.wav
==piceas-050824-083400.wav
==piceas-050824-083600.wav
==piceas-050824-083800.wav
==piceas-050824-084000.wav
Total sound length: 2236.179
Number of whistleness frames: 1096/2215(str(num_whistle/num_frames))
Number of training data samples: 1762

/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/PICEAS2005_48kHz/bottlenose a73
==piceas-050820-094000-chan1.wav
==piceas-050820-094500-chan1.wav
==piceas-050820-095000-chan1.wav
=

In [30]:
feature_train = np.stack(spec_feas_train)
del spec_feas_train
label_train = np.array(label_train)
feature_test = np.stack(spec_feas_test)
del spec_feas_test
label_test = np.array(label_test)

In [31]:
ten_sec_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__dataset/ten_seconds_data'
np.savez(os.path.join(ten_sec_path, sound_deploy[dd]), feature_train=feature_train, label_train=label_train, feature_test=feature_test, label_test=label_test)

In [32]:
species_to_code[os.path.basename(ee).split(' ')[0]]

'STR'

In [33]:
spec_feas.shape

(3094, 100, 128, 1)

In [34]:
step_ind

10

### # For the whole encounter
#     # ave score
#     label_pred = np.sqrt(((scores**2.0)*(np.multiply(whistleness_bin.reshape(-1, 1), np.ones((1,8))))*((scores>0.50))))  # .mean(axis=0)
#     num_large_score =(label_pred>0.0).sum(axis=0)
#     total_score = label_pred.sum(axis=0)
#     avg_score = np.zeros(8)
#     for cc in range(8):
#         if num_large_score[cc] > 0:
#             avg_score[cc] = total_score[cc] / num_large_score[cc]
#     avg_score_list.append(avg_score)
#     print('average score:')
#     print(avg_score)
    
#     # count_large = (scores>0.70).sum(axis=0)
#     print('count_large:')
#     print(num_large_score)

            
            
#             species_10s_pred.append(np.argmax(total_score, axis=0))
#             species_10s_truth.append(species_dict[species_to_code[ee.split(" ")[0]]])

        
#         label_pred_curr = np.sqrt(((score_curr**2.0)*(np.multiply(whistle_bin_curr.reshape(-1, 1), np.ones((1,8))))*((score_curr>0.50))))  # .mean(axis=0)
#         num_large_score =(label_pred_curr>0.0).sum(axis=0)
#         total_score = label_pred_curr.sum(axis=0)
#         avg_score_curr = np.zeros(8)
#         for cc in range(8):
#             if num_large_score[cc] > 0:
#                 avg_score_curr[cc] = total_score[cc] / num_large_score[cc]
#         print('average score:')
#         print(avg_score_curr)
        
#         # count_large = (scores>0.70).sum(axis=0)
#         print('count_large:')
#         print(num_large_score)